In [139]:
import pickle
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import html5lib

In [67]:
df_artista_score = pd.read_pickle('df_artista_score.pkl')

In [68]:
df_artista_score.head()

,artista,num_acordes_distintos_decil,perc_acordes_fora_campo_decil,tamanho_medio_progressao_decil,media_raridade_decil,media_complexidade_decil,qtde_musicas_decil,qtde_geral_acordes_decil,nota_harmonia,num_palavras_distintas_letra_decil,tf_idf_sum_decil,num_rimas_decil,num_rimas_classes_diferentes_decil,vocabulario_geral_decil,nota_letra,nota_final
0,100-parea,3.0,1.0,2.0,2.0,1.0,3,2,2.000000,10.0,10.0,9.0,9.0,6.0,8.8,5.400000
1,14-bis,8.0,5.0,5.0,8.0,6.0,8,10,7.142857,5.0,6.0,4.5,4.5,8.0,5.6,6.371429
2,1kilo,1.0,6.0,8.0,6.0,8.5,4,5,5.500000,10.0,10.0,10.0,10.0,10.0,10.0,7.750000
3,2-minutos,3.0,7.0,6.0,4.0,1.0,9,2,4.571429,1.0,3.0,1.0,2.0,5.0,2.4,3.485714
4,3-palavrinhas,3.0,1.0,2.0,2.0,1.0,7,4,2.857143,1.0,1.0,5.0,3.5,3.0,2.7,2.778571


In [132]:
def list_unique(lista):
    lista_nova = []
    for i in lista:
        if i not in lista_nova:
            lista_nova.append(i)
    return lista_nova

df_vagalume_albuns = pd.DataFrame()
df_vagalume_discografia = pd.DataFrame()

count = 0
for artista in list(df_artista_score['artista']):
    url = 'https://www.vagalume.com.br/' + artista + '/discografia/'
    page = requests.get(url)
    html_doc = page.text
    soup = BeautifulSoup(html_doc, 'html5lib')
    if 'Ops... 404' not in soup.text:
        lista_musicas_albuns = [k.get('href') for k in soup.find_all('a') if 'album:' in k.get('href')]
        lista_albuns = [k.split(':album:')[-1] for k in lista_musicas_albuns]
        lista_musicas = [k.split(artista)[1].split('.html#play')[0].replace('/', '') for k in lista_musicas_albuns]
        df_discografia = pd.DataFrame({'artista': [artista]*len(lista_musicas_albuns), 'album': lista_albuns, 'musica': lista_musicas})
        lista_ano_album = [[artista, k.find(class_='albumYear').text, k.find('a').get('href').split('/')[-1].replace('.html', '')] for k in soup.find_all(class_='cardAlbumInfos')]
        df_albuns = pd.DataFrame(lista_ano_album).rename(columns={0: "artista", 1: "ano", 2: "album"})
        df_vagalume_albuns = pd.concat([df_vagalume_albuns, df_albuns])
        df_vagalume_discografia = pd.concat([df_vagalume_discografia, df_discografia])
    
    if count%10==0:
        print(count, artista)
    count+=1

0 100-parea
10 a-pequena-sereia
20 aditive
30 alceu-valenca
40 altermar-dutra
50 anderson-freire
60 asa-de-aguia
70 baitaca
80 banda-magnificos
90 belchior
100 bin
110 bruno-e-marrone
120 capiba
130 cartola
140 celso-fonseca
150 chiclete-com-banana
160 cicero
170 claudionor-germano
180 cristiano-neves
190 daniela-mercury
200 detonautas
210 diogo-nogueira
220 drops
230 eduardo-costa
240 enygma
250 fabio-jr
260 ferrugem
270 forro-boys
280 gabriel-diniz
290 garotos-podres
300 gildo-de-freitas
310 grupo-bom-gosto
320 guilherme-e-benuto
330 helena-de-lima
340 imaginasamba
350 ivete-sangalo
360 jeito-moleque
370 joao-mineiro-e-marciano
380 jorge-mateus
390 jovelina-perola-negra
400 kid-abelha
410 laura-souguellis
420 leonardo-andre
430 los-hermanos
440 luiz-carlos-magno
450 mamonas-assassinas
460 marcelo-jeneci
470 maria-marcal
480 mastruz-com-leite
490 mc-davi
500 mc-kevin
510 mc-pedrinho
520 milton-nascimento
530 moraes-moreira
540 nando-reis
550 nelson-ned
560 nx-zero
570 os-bertussi
580 

In [133]:
len(df_vagalume_discografia)

73571

In [134]:
df_vagalume_discografia.head()

,artista,album,musica
0,14-bis,novo-millennium-14-bis,linda-juventude
1,14-bis,novo-millennium-14-bis,linda-juventude
2,14-bis,novo-millennium-14-bis,planeta-sonho
3,14-bis,novo-millennium-14-bis,siga-o-sol
4,14-bis,novo-millennium-14-bis,carrossel


In [135]:
df_vagalume_albuns.head()

,artista,ano,album
0,14-bis,2005,novo-millennium-14-bis
1,14-bis,2005,serie-bis-14-bis
2,14-bis,2004,outros-planos
3,14-bis,1999,bis-acustico
4,14-bis,1996,siga-o-sol


In [136]:
df_vagalume_discografia_ = df_vagalume_discografia.merge(df_vagalume_albuns, on = ['artista', 'album'], how = 'inner')

In [137]:
df_vagalume_discografia_.head()

,artista,album,musica,ano
0,14-bis,novo-millennium-14-bis,linda-juventude,2005
1,14-bis,novo-millennium-14-bis,linda-juventude,2005
2,14-bis,novo-millennium-14-bis,planeta-sonho,2005
3,14-bis,novo-millennium-14-bis,siga-o-sol,2005
4,14-bis,novo-millennium-14-bis,carrossel,2005


In [138]:
df_vagalume_discografia_.to_pickle('df_vagalume_discografia_.pkl')